# 0. Loading Data

In [ ]:
import pandas as pd
import re
from datetime import datetime
from meteostat import Point, Daily
from sodapy import Socrata
import matplotlib.pyplot as plt



pd.set_option('display.max_columns', None)

In [ ]:
import os
print(os.getcwd())

In [ ]:
# CONSTANTS
DATASET_ID = "erm2-nwe9"  # NYC 311 Service Requests ID
APP_TOKEN = None          # Optional: Sign up for an app token at NYC Open Data for higher limits

def get_311_data(start_date="2023-01-01", end_date="2025-01-01", limit=10000):
    """
    Fetches 311 data using the Socrata API.
    """
    client = Socrata("data.cityofnewyork.us", APP_TOKEN)
    
    # SoQL Query: Filter by date to avoid crashing memory
    # We select key columns to keep the dataframe lean
    results = client.get(
        DATASET_ID,
        where=f"created_date > '{start_date}' AND created_date < '{end_date}'",
        select="unique_key, created_date, complaint_type, latitude, longitude, borough, location_type",
        limit=limit
    )
    
    df = pd.DataFrame.from_records(results)
    
    # Convert dates to datetime objects immediately
    df['created_date'] = pd.to_datetime(df['created_date'])
    df['closed_date'] = pd.to_datetime(df['closed_date'])
    
    return df

In [ ]:
def get_nyc_weather(start_year, end_year):
    # Coordinates for NYC (Central Park area)
    location = Point(40.7831, -73.9712)

    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)

    # Fetch daily data
    data = Daily(location, start, end)
    data = data.fetch()
    
    return data

In [ ]:
df_311 = get_311_data(start_date="2023-01-01", limit=7000000)

In [ ]:
df_weather = get_nyc_weather(2023, 2024)

# 1. Cleaning Data

In [ ]:
df_311 = df_311[df_311.latitude.notnull()]
df_311 = df_311[df_311.longitude.notnull()]
df_311 = df_311[df_311.descriptor.notnull()]

In [ ]:
df_311.complaint_type = df_311.complaint_type.str.title()

In [ ]:
def update_OSE(agency):

  """Takes in an agency name and converts to abbreviated form, if the agency is
     the Office of Special Enforcement."""

  if agency == 'MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT':
      return "OSE"
  else:
      return agency
    
df_311.agency = df_311.agency.map(lambda agency: update_OSE(agency))

In [ ]:
def update_noise(complaint):

  """Takes in an complaint name and updates it, if it is
     an unspecified noise complaint."""

  if complaint == 'Noise':
    
    return "Noise - Unspecified"

  else:

    return complaint
    
df_311.complaint_type = df_311.complaint_type.map(
    lambda complaint: update_noise(complaint))

In [ ]:
df_311['day'] = [str(i.date()) for i in df_311.created_date]
df_311['month'] = [int(i.month) for i in df_311.created_date]
df_311['day_of_week'] = [int(i.weekday()) for i in df_311.created_date]
df_311['hour'] = [int(i.hour) for i in df_311.created_date]

In [ ]:
num_pattern = r'[-()0-9]'
df_311.descriptor = df_311.descriptor.map(lambda x: re.sub(num_pattern, '', x).lower())
df_311.descriptor = df_311.descriptor.map(lambda x: re.sub('/', ' ', x))

In [ ]:
df_merged = df_311.merge(
    df_weather, 
    left_on=df_311['created_date'].dt.date, # Extracts python date object
    right_on=df_weather['time'].dt.date
)

# 2. Removing rare complaint types

In [ ]:
counts = df_merged['complaint_type'].value_counts()
total_rows = len(df_merged)

# 2. Define Thresholds to compare
thresh_1pct = total_rows * 0.01   # 1% (Your proposal)
thresh_01pct = total_rows * 0.002 # 0.1% (Standard alternative)

print(f"Total Rows: {total_rows:,}")
print(f"1% Threshold: {int(thresh_1pct):,} rows")
print(f"0.2% Threshold: {int(thresh_01pct):,} rows")

# 3. Identify what gets dropped
drop_1pct = counts[counts < thresh_1pct]
drop_01pct = counts[counts < thresh_01pct]

print(f"\n--- At 1% Threshold ({int(thresh_1pct):,} rows) ---")
print(f"You would drop {len(drop_1pct)} unique complaint types.")
print(f"Examples of dropped types: {drop_1pct.index[:5].tolist()}")
print(f"Total data lost: {drop_1pct.sum():,} rows ({(drop_1pct.sum()/total_rows)*100:.2f}%)")

print(f"\n--- At 0.2% Threshold ({int(thresh_01pct):,} rows) ---")
print(f"You would drop {len(drop_01pct)} unique complaint types.")
print(f"Total data lost: {drop_01pct.sum():,} rows ({(drop_01pct.sum()/total_rows)*100:.2f}%)")

# 4. Visual "Elbow" Plot
# We want to see where the curve flattens out
cumulative_percent = counts.cumsum() / total_rows
plt.figure(figsize=(10, 5))
plt.plot(range(len(counts)), cumulative_percent.values, marker='o', markersize=3)
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Data Coverage')
plt.title('Cumulative Distribution of Complaint Types')
plt.xlabel('Number of Complaint Types (Ranked by Volume)')
plt.ylabel('Cumulative % of Dataset')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
threshold_count = len(df_merged) * 0.002
counts = df_merged['complaint_type'].value_counts()

high_volume_types = counts[counts >= threshold_count].index.tolist()

whitelist = [
    'Snow Or Ice', 
    'Street Condition', 
    'Damaged Tree', 
    'Heat/Hot Water', 
    'Water Leak', 
    'Standing Water', 
    'Sewer'
]

final_types_to_keep = list(set(high_volume_types + whitelist))

df_merged_reduced = df_merged[df_merged['complaint_type'].isin(final_types_to_keep)].copy()

print(f"Original Rows: {len(df_merged):,}")
print(f"Filtered Rows: {len(df_merged_reduced):,}")
print(f"Data Retained: {(len(df_merged_reduced)/len(df_merged))*100:.2f}%")
print(f"Unique Complaint Types: {df_merged_reduced['complaint_type'].nunique()}")

In [ ]:
df_merged_reduced = df_merged[df_merged.groupby('complaint_type').complaint_type.transform('count')>13005].copy() 

In [ ]:
df_merged_reduced = df_merged_reduced.dropna()

# 3. Saving data

In [ ]:
project_dir = "."

In [ ]:
df_311.to_csv(f"{project_dir}/data/311_data.csv")

In [ ]:
df_weather.to_csv(f"{project_dir}/data/weather_data.csv")

In [ ]:
df_merged.to_csv(f"{project_dir}/data/merged_data.csv")

In [ ]:
df_merged_reduced.to_csv(f"{project_dir}/data/merged_reduced_data.csv")